In [ ]:
pip install pytrends

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import re
import os
from pandas.io.json import json_normalize
from datetime import date,datetime,timedelta
import time
filename = 'KW_HSBC_jun22.csv'
proyecto = 'HSBC - ' # El nombre del proyecto se debe ir cambiando y representa la marca para la que se trabaja.
periodos = ['2019-06-01 2019-06-30', '2020-06-01 2020-06-30', '2021-06-01 2021-06-30', '2022-04-16 2022-04-30']
# El periodo se visualiza por AÑO / MES / DIA, de punto "A" a punto "B".

def categoria(cats):
    switcher={
        '7':'Finance',    
        '18':'Shopping',
        '37': 'Banking',
        '279': 'Credit & Lending',
        '468': 'Auto Financing',
        '811': 'Credit Cards',
        '38': 'Insurance',
        '107': 'Investing',
        '44':'Beauty & Fitness',
        '8':'Games',
        '45':'Health',
        '65':'Hobbies & Leisure',
        '299':'Online Communities',
        '14':'People & Society',
        '20':'Sports',
        '0': 'Global',
        '35': 'Music & Audio',
        '71': 'Food & Drink',
        '560' :'Non-Alcoholic Beverages',
        '442': 'Ecology & Environment',
        '1307': 'Recycling',
        '1349': 'Water Supply & Treatment',
        '3': 'Arts & Entertainment',
        '34': 'Movies',
        '36': 'TV & Video',
        '185': 'Fashion & Style',
        '22': 'Books & Literature',
        '799': 'Business Education',
        '5': 'Computers & Electronics',
        '899': 'Game Systems & Consoles',
        '1290': 'Family-Oriented Games & Activities',
        '105': 'Online Games',
        '195': 'Reproductive Health',
        '257': 'Substance Abuse',
        '66': 'Pets & Animals',
        '977': 'Special Occasions',
        '11': 'Home & Garden',
        '1131': 'Family & Relationships',
        '154': 'Kids & Teens',
        '70': 'Gifts & Special Event Items',
        '67': 'Travel'
        }
    return switcher.get(cats,"Categoria Invalida")
interese = pd.DataFrame(columns=('dates','keyword','interes', 'hour','dateq'))
regions = pd.DataFrame(columns=('dates','keyword','interes', 'hour','location'))
big_trend_brand = pd.DataFrame(columns=('query','interes','keyword','nquery','hour'))

pytrend = TrendReq(timeout=(45),retries=2)
now = datetime.now()- timedelta(hours=5)
hour = now.strftime('%H:%M')
day = now.strftime('%Y-%m-%d')

#periodos = ['2017-01-01 2017-12-31','2018-01-01 2018-12-31','2019-01-01 2019-12-31','2020-01-01 2020-12-31','2021-01-01 2021-12-31']
#periodos = ['2021-04-01 2021-04-30','2021-05-01 2021-05-31','2021-11-01 2021-11-30', '2022-01-01 2022-02-15']
#periodos = ['2022-01-01 2022-01-31', '2022-02-01 2022-02-28', '2022-03-01 2022-03-31']
#periodos = ['2010-01-01 2010-12-31', '2011-01-01 2011-12-31', '2012-01-01 2012-12-31', '2013-01-01 2013-12-31', '2014-01-01 2014-12-31']
#periodos = ['2015-01-01 2015-12-31', '2016-01-01 2016-12-31', '2017-01-01 2017-12-31', '2018-01-01 2018-12-31', '2019-01-01 2019-12-31']
#periodos = ['2020-01-01 2020-12-31', '2021-01-01 2021-12-31', '2022-01-01 2022-04-15']
#periodos = ['2010-06-01 2010-07-31', '2014-06-01 2014-07-31', '2018-06-01 2018-07-31']
# El periodo se visualiza por AÑO / MES / DIA, de punto "A" a punto "B".

#Búsquedas desde una lista en un archivo csv. se carga de la siguiente forma:

dfnombres=pd.read_csv(filename)
dfnombres['Producto']=dfnombres['Producto']. apply(lambda x : x.strip())
#dfnombres['Producto']=dfnombres['Producto']. apply(lambda x : x.split(' ')[0])
solonombres=dfnombres.drop_duplicates(subset=['Producto'])
categorias = ['0'] #COVID
#groups = [solonombres['Producto'][i:i + 5] for i in range(0, len(solonombres), 5)] #Para correr de 5 en 5
groups = [solonombres['Producto'][i:i + 1] for i in range(0, len(solonombres), 1)] #Para correr de 1 en 1


#Búsquedas desde una lista personalizada:

#ListaKeywords  = ['Play Doh','Fud','Leche Entera','Zwan','Santa Clara', 'Deslactosada', 'Alpura', 'Lala', 'Leche', 'Jamón', 'Leche Lala', 'Lala 100', 'Alpura 100']
#ListaKeywords  = ['Juego Nintendo Switch']
#categorias = ['0'] #COVID
#groups = [ListaKeywords[i:i + 5] for i in range(0, len(ListaKeywords), 5)]

for kw_list in groups:
    for period in periodos:
        try:
            pytrend.build_payload(kw_list = kw_list, timeframe = period,geo='MX')
            #pytrend.build_payload(kw_list = kw_list, timeframe = period,geo='US')
            interest_over_time_df = pytrend.interest_over_time()
            interes = pd.DataFrame(columns=('dates','keyword','interes', 'hour','dateq','marca'))
            for kw in kw_list:
                interes.dates = interest_over_time_df.index
                interes.keyword = kw
                interes.interes = interest_over_time_df[kw].values
                interes.hour = hour
                interes.dateq = day
                interes.marca = proyecto
                interese = pd.concat([interese,interes], axis=0, ignore_index=True)
            interest_by_region_df = pytrend.interest_by_region(inc_low_vol=True, inc_geo_code=False)

            #
            state = pd.DataFrame(columns=('dates','keyword','interes', 'hour','location','marca'))
            for kw in kw_list:
                state.interes = interest_by_region_df[kw].values
                state.location = interest_by_region_df.index
                state.hour = hour
                state.dates = day
                state.keyword = kw
                state.marca = proyecto
                regions = pd.concat([regions,state], axis=0, ignore_index=True)
            #
            for cats in categorias:
                #REALIZA LA CONSULTA A GOOGLE TRENDS CON LOS PARAMETROS ESTABLECIDOS
                pytrend.build_payload(kw_list,geo = 'MX', timeframe = period, cat = cats)
                #pytrend.build_payload(kw_list,geo = 'US', timeframe = period, cat = cats)
                trends= pytrend.related_queries()
                #DEVUELVE UN JSON {"KEYWORD":{"TOP:QUERY,VALUE" , "RISIGN:QUERY,VALUE"}}
                data1 = pd.DataFrame(columns=('query','value','keyword'))
                data2 = pd.DataFrame(columns=('query','value','keyword'))

                for kw in kw_list:  
                    datat = pd.DataFrame(trends[kw]['top'])
                    datar = pd.DataFrame(trends[kw]['rising'])
                    if datat.empty is False or datar.empty is False: 
                        datat['keyword'] = kw
                        datar['keyword'] = kw
                        data1 = pd.concat([data1,datat], axis=0, ignore_index=True, sort=False)
                        data2 = pd.concat([data2,datar], axis=0, ignore_index=True, sort=False)

                data1 = data1.rename(columns={'value':'interes'})
                data2 = data2.rename(columns={'value':'nquery'})

                trend_brand = pd.merge(data1, data2, how='outer', on='query')
                trend_brand.keyword_x.fillna(trend_brand.keyword_y,  inplace=True)
                trend_brand = trend_brand.rename(columns={'keyword_x':'keyword'})
                trend_brand = trend_brand.fillna(0)
                trend_brand = trend_brand.drop(['keyword_y'], axis=1)

                trend_brand['hour'] = hour
                trend_brand['dates'] = day
                trend_brand['periodo'] = period
                trend_brand['categoria'] = categoria(cats)
                trend_brand['marca'] = proyecto
                namef = 'GT_KW'+categoria(cats)+period+'.csv'
                big_trend_brand = big_trend_brand.append(trend_brand, ignore_index = True)
                ###importacion CSV
                #export_csv = trend_brand.to_csv (namef, index = None, header=True , encoding='UTF-8')
                print(categoria(cats),kw_list)
                print(period)
                print(hour)

                    ###importacion a ADW
                    #ntable = "STG_BS_GT_BRANDS"
                    #data = trend_brand.astype(str)
                    #colus = ', '.join([':'+str(x+1) for x in range(len(data.columns.tolist()))])
                    #sql='insert into '+ntable+' values('+colus+')'
                    #print(sql)

                    #df_list = data.values.tolist()
                    #n = 0
                    #for i in data.iterrows():
                    #    cursor.execute(sql,df_list[n])
                    #    connection.commit()
                    #    n += 1

                time.sleep(10)
        except:
            pass

    #big_trend_brand  
    #export_csv = trend_brand.to_csv (namef, index = None, header=True , encoding='UTF-8')

interese.to_csv('{}_Intereses_{}_{}.csv'.format(proyecto, day, hour), encoding='utf-8')
regions.to_csv('{}_Regiones_{}_{}.csv'.format(proyecto, day, hour), encoding='utf-8')
big_trend_brand.to_csv('{}_Related_{}_{}.csv'.format(proyecto, day, hour), encoding='utf-8')

Global 0    cuenta de ahorro
Name: Producto, dtype: object
2019-06-01 2019-06-30
13:50
Global 0    cuenta de ahorro
Name: Producto, dtype: object
2020-06-01 2020-06-30
13:50
Global 0    cuenta de ahorro
Name: Producto, dtype: object
2021-06-01 2021-06-30
13:50
Global 0    cuenta de ahorro
Name: Producto, dtype: object
2022-04-16 2022-04-30
13:50
Global 1    cuenta de debito
Name: Producto, dtype: object
2019-06-01 2019-06-30
13:50
Global 1    cuenta de debito
Name: Producto, dtype: object
2020-06-01 2020-06-30
13:50
Global 1    cuenta de debito
Name: Producto, dtype: object
2021-06-01 2021-06-30
13:50
Global 1    cuenta de debito
Name: Producto, dtype: object
2022-04-16 2022-04-30
13:50
Global 2    tarjeta de debito
Name: Producto, dtype: object
2019-06-01 2019-06-30
13:50
Global 2    tarjeta de debito
Name: Producto, dtype: object
2020-06-01 2020-06-30
13:50
Global 2    tarjeta de debito
Name: Producto, dtype: object
2021-06-01 2021-06-30
13:50
Global 2    tarjeta de debito
Name: Prod

In [ ]:
interese

,dates,keyword,interes,hour,dateq,marca
0,2022-04-01,Batman,56,09:33,2022-04-22,SUPERS
1,2022-04-02,Batman,76,09:33,2022-04-22,SUPERS
2,2022-04-03,Batman,89,09:33,2022-04-22,SUPERS
3,2022-04-04,Batman,65,09:33,2022-04-22,SUPERS
4,2022-04-05,Batman,52,09:33,2022-04-22,SUPERS
5,2022-04-06,Batman,56,09:33,2022-04-22,SUPERS
6,2022-04-07,Batman,60,09:33,2022-04-22,SUPERS
7,2022-04-08,Batman,56,09:33,2022-04-22,SUPERS
8,2022-04-09,Batman,84,09:33,2022-04-22,SUPERS
9,2022-04-10,Batman,100,09:33,2022-04-22,SUPERS


In [ ]:
regions

,dates,keyword,interes,hour,location,marca
0,2022-04-22,Batman,78,09:33,Aguascalientes,SUPERS
1,2022-04-22,Batman,84,09:33,Baja California,SUPERS
2,2022-04-22,Batman,97,09:33,Baja California Sur,SUPERS
3,2022-04-22,Batman,68,09:33,Campeche,SUPERS
4,2022-04-22,Batman,60,09:33,Chiapas,SUPERS
...,...,...,...,...,...,...
91,2022-04-22,Goku,58,09:33,Tamaulipas,SUPERS
92,2022-04-22,Goku,74,09:33,Tlaxcala,SUPERS
93,2022-04-22,Goku,67,09:33,Veracruz,SUPERS
94,2022-04-22,Goku,59,09:33,Yucatan,SUPERS


In [ ]:
big_trend_brand

,query,interes,keyword,nquery,hour,dates,periodo,categoria,marca
0,the batman,100,Batman,0,09:33,2022-04-22,2022-04-01 2022-04-15,Global,SUPERS
1,batman 2022,17,Batman,0,09:33,2022-04-22,2022-04-01 2022-04-15,Global,SUPERS
2,batman cinepolis,15,Batman,0,09:33,2022-04-22,2022-04-01 2022-04-15,Global,SUPERS
3,pelicula batman,11,Batman,0,09:33,2022-04-22,2022-04-01 2022-04-15,Global,SUPERS
4,batman hbo,11,Batman,0,09:33,2022-04-22,2022-04-01 2022-04-15,Global,SUPERS
...,...,...,...,...,...,...,...,...,...
84,pastel de goku,0,Goku,110,09:33,2022-04-22,2022-04-01 2022-04-15,Global,SUPERS
85,goku super saiyan 3,0,Goku,110,09:33,2022-04-22,2022-04-01 2022-04-15,Global,SUPERS
86,fotos de goku black,0,Goku,80,09:33,2022-04-22,2022-04-01 2022-04-15,Global,SUPERS
87,goku fase 100,0,Goku,80,09:33,2022-04-22,2022-04-01 2022-04-15,Global,SUPERS


In [ ]:
interese.to_csv('{}_Intereses_{}_{}.csv'.format(proyecto, day, hour), encoding='utf-8')

regions.to_csv('{}_Regiones_{}_{}.csv'.format(proyecto, day, hour), encoding='utf-8')

big_trend_brand.to_csv('{}_Related_{}_{}.csv'.format(proyecto, day, hour), encoding='utf-8')

In [ ]:
pwd

'/Users/raul.castillo/SCRIPTS'

In [ ]:
groups = [kw_list[i:i + 5] for i in range(0, len(kw_list), 5)]
groups

[['Play Doh', 'plastilina patra niños', 'plastilina no toxica']]

In [ ]:
pytrend.trending_searches(pn='mexico')

NameError: name 'pytrend' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=705432f3-5a59-4b7f-9d5a-243608199265' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>